In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import make_classification
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from skl2onnx import convert_sklearn
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
import goodModel

import pandas as pd
import numpy as np

In [ ]:
def modify_data(df):
    print("Starting data augmentation")

    df = goodModel.drop_personality_columns(df)
    print("Data augmentation dropping personality columns complete")
    df = goodModel.data_augmentation_age(df)
    print("Data augmentation age complete")

    df = goodModel.drop_taaleis_columns(df)
    print("Data augmentation dropping taaleis complete")

    df = goodModel.change_labels_data_augmentation_binary(df, 'persoon_geslacht_vrouw')
    print("Data augmentation gender complete")


    #df = goodModel.data_augmentation_neighborhoods(df)
    return df

In [ ]:
df = pd.read_csv('data/investigation_train_large_checked.csv')

In [ ]:
data = modify_data(df)

# Let's specify the features and the target
y = data['checked']
X = data.drop(['checked', 'Ja', 'Nee'], axis=1)
X = X.astype(np.float32)

In [ ]:
from sklearn.ensemble import RandomForestClassifier


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessing
numeric_features = list(range(X.shape[1]))  # assuming all features are numeric in make_classification
numeric_transformer = StandardScaler()

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
    ]
)

# Pipelines for both models
linreg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier",  RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42))
])

# Train both models on full training data
linreg_pipeline.fit(X_train, y_train)


In [ ]:
def pred_to_binary(predictions, threshold=0.5):
    """Converts risk scores to binary values."""
    return (predictions >= threshold).astype(int)

In [ ]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    linreg_pipeline, initial_types=[('X', FloatTensorType((None, X.shape[1])))],
    target_opset=12)

# How to get metrics such as accuracy
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})
y_pred_onnx_binary = pred_to_binary(y_pred_onnx[0]) # Do not forget
y_test = y_test.astype(int) # Do not forget
accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx_binary)

print('Accuracy of the ONNX model: ', accuracy_onnx_model)

# Let's save the model
onnx.save(onnx_model, "model/model_2.onnx")